### Fine-Tune Command

In [1]:
!pip install transformers[torch] datasets peft bitsandbytes tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 373.5 kB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 268.5 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 1.2 MB/s eta 0:00:00ta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 904.9 kB/s eta 0:00:00a 0:00:01
INFO: pip is looking at multiple versions of huggingface-hub to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of huggingface-hub to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip

In [121]:
import json

curated_ds = json.load(open("data_5000.json"))

In [180]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "EleutherAI/pythia-160m"

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.add_special_tokens(
    {"additional_special_tokens": ["### End", "### Instruction:", "### Response:\n", "### Context:"]}
)

model = AutoModelForCausalLM.from_pretrained(model_id, revision="step130000")
# model = AutoModelForCausalLM.from_pretrained(model_id, revision="step143000")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [181]:
def tokenize_alpaca_item(tokenizer, item):
    text = f'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction: Rewrite this as a command in less than 10 words for a human, do not provide anything else: {item["original_instruction"]} \n\n### Response:\n{item["command"]}\n\n### End### End### End'
    return tokenizer(text)

In [182]:
import tqdm

ds_tokenized = [tokenize_alpaca_item(tokenizer, item) for item in tqdm.tqdm(curated_ds["data"])]


100%|██████████| 5000/5000 [00:01<00:00, 3438.11it/s]


In [183]:
for tmp in tqdm.tqdm(ds_tokenized):
    # print(tmp['input_ids'][-1])
    # print(tokenizer("### End"))
    assert tmp['input_ids'][-1]==tokenizer("### End")['input_ids'][0]

100%|██████████| 5000/5000 [00:00<00:00, 23017.30it/s]


In [184]:
from peft import LoraConfig, get_peft_model, PeftModel

def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

config = LoraConfig(
    r=8, 
    lora_alpha=32, 
    target_modules=["query_key_value"], 
    lora_dropout=0.05, 
    bias="none", 
    task_type="CAUSAL_LM",
) 

peft_model = get_peft_model(model, config)
print_trainable_parameters(peft_model)

trainable params: 294912 || all params: 162617856 || trainable%: 0.18135277838123753


In [185]:
import transformers

# needed for gpt-neo-x tokenizer
tokenizer.pad_token = tokenizer.eos_token

trainer = transformers.Trainer(
    model=model,
    train_dataset=ds_tokenized,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=4,
        gradient_accumulation_steps=4,
        warmup_steps=2,
        # max_steps=100,
        num_train_epochs=1,
        learning_rate=3e-4,
        fp16=True,
        logging_steps=5,
        output_dir="outputs",
        optim="paged_adamw_8bit"
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss
5,47.492100
10,41.572500
15,20.536000
20,9.264400
25,4.306400
30,3.425300
35,2.657000
40,2.476900
45,2.171100
50,2.010500


TrainOutput(global_step=156, training_loss=5.417370505822011, metrics={'train_runtime': 199.9403, 'train_samples_per_second': 25.007, 'train_steps_per_second': 0.78, 'total_flos': 291855272435712.0, 'train_loss': 5.417370505822011, 'epoch': 0.9984})

In [155]:
PROMPT = "Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\ What are tips to stay healthy? \n\n### Response:\n"
answer = tokenizer.decode(peft_model.to('cuda').generate(**tokenizer(PROMPT, return_tensors="pt").to('cuda'), max_new_tokens=50, temperature=0.99, top_k=500, do_sample=True)[0])


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


"Stay energetic." And say 8 kinds of breakfast meals.

"Younggy's breakfasts are good for more than just breakfast."



                   
                   
  ิياياыхалитироваться конечно сл


In [195]:
item = curated_ds["data"][30]
print(item)
PROMPT = f'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction: Rewrite this as a command in less than 10 words for a human, do not provide anything else: {item["original_instruction"]} \n\n### Response:\n'

answer = tokenizer.decode(peft_model.to('cuda').generate(**tokenizer(PROMPT, return_tensors="pt").to('cuda'), max_new_tokens=50)[0])
print(answer.split("### End")[0].split("### Response:\n")[1].split('\n')[0])

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


{'command': 'Explain how plants make energy from sunlight.', 'search_term': '"cellular respiration in plant cells"', 'explanation': 'Cellular respiration in plant cells occurs through the process of photosynthesis, where light energy is converted into chemical energy in the form of glucose, followed by cellular respiration to break down glucose and produce ATP.', 'instruction_prompt': '### Instruction: Explain how plants make energy from sunlight. \n\n### Context: Cellular respiration in plant cells occurs through the process of photosynthesis, where light energy is converted into chemical energy in the form of glucose, followed by cellular respiration to break down glucose and produce ATP. \n\n### Response:\n', 'original_instruction': 'Explain the process of cellular respiration in plants.', 'original_input': '', 'original_output': "Cellular respiration in plants is a complex process in which the plant converts glucose into usable energy. The process begins when the plant takes in car

In [197]:
peft_model.save_pretrained("v1_pythia_160m_step130000-command")

In [201]:
new = AutoModelForCausalLM.from_pretrained("v1_pythia_160m_step130000-command")

item = curated_ds["data"][0]
print(item)
PROMPT = f'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction: Rewrite this as a command in less than 10 words for a human, do not provide anything else: {item["original_instruction"]} \n\n### Response:\n'

answer = tokenizer.decode(new.to('cuda').generate(**tokenizer(PROMPT, return_tensors="pt").to('cuda'), max_new_tokens=50)[0])
print(answer.split("### End")[0].split("### Response:\n")[1].split('\n')[0])

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


{'command': '"Maintain regular exercise, eat balanced meals, and get enough sleep."', 'search_term': '"Staying Healthy Tips"', 'explanation': 'To maintain optimal physical and mental well-being, it is essential to incorporate simple yet effective habits into your daily routine, such as drinking plenty of water, exercising regularly, getting enough sleep, and maintaining a balanced diet.', 'instruction_prompt': '### Instruction: "Maintain regular exercise, eat balanced meals, and get enough sleep." \n\n### Context: To maintain optimal physical and mental well-being, it is essential to incorporate simple yet effective habits into your daily routine, such as drinking plenty of water, exercising regularly, getting enough sleep, and maintaining a balanced diet. \n\n### Response:\n', 'original_instruction': 'Give three tips for staying healthy.', 'original_input': '', 'original_output': '1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. \n2. Exercise regularly t